Human generated data was collected from Wikipedia, Aljazeera News and Books from Gutenberg using BeautifulSoup Library.

1909 samples came from Wikipedia, 243 from Aljazeera News and 3000 from Gutenberg.

For Machine Generated text gpt-2 was used through Huggingface library. Created a mixture of templates, topics and categories to generate random prompts. Target was to collect 6000 responses but runtime disconnected after collecting 5580 Samples only. (Took about 9 hours to collect this, didn’t have energy nor patience to restart the process so settled with 5580 only).

The data file initially had length of 781,200 but after removing duplicates it came out as 5580 only.
Did some cleaning after getting the complete data of 10244 records, removed links, non, alpha-numeric characters etc. Also, made sure of suitable number of words in the machine generate dataset at the minimum of 74.

The complete data was saved in “final_combined_dataset.csv”. (will upload it along with the assignment)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/NLP CLASS/A 1") # Replace with the actual path
print(os.getcwd())

/content/drive/MyDrive/NLP CLASS/A 1


In [4]:
!pip install wikipedia-api
!pip install pandas
import random
import re
import pandas as pd
import wikipediaapi

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15384 sha256=551ad2bb8b2f6cd1d6b37b3a7d77bcc2f5623c362324f3812cf924f6fd910dcf
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


In [6]:
wiki_wiki = wikipediaapi.Wikipedia(
    language="en",
    user_agent="MyWikipediaScraper/1.0 (mailto:ankitchahal27@gmail.com)"
)

In [8]:
topics = ["Artificial intelligence", "Climate change", "History of mathematics",
    "Quantum mechanics", "Shakespeare", "Modern architecture",
    "Neural networks", "Renaissance art", "Human evolution",
    "Space exploration", "Blockchain", "Psychology", "Machine learning",
    "Cybersecurity", "Ancient civilizations", "Philosophy of science",
    "Cryptography", "Music theory", "World War II", "Genetics",
    "Medical advancements", "Astronomy", "Linguistics", "Sociology",
    "Economics", "Political philosophy", "Feminism", "Robotics",
    "Environmental ethics", "Game theory"]

In [9]:
# Function to clean and extract text
def clean_wiki_text(text):
    text = re.sub(r'\n+', ' ', text)  # Remove newlines
    text = re.sub(r'\[[^\]]*\]', '', text)  # Remove references like [1], [2]
    text = re.sub(r'\([^)]*\)', '', text)  # Remove text in parentheses
    text = re.sub(r'==.*?==', '', text)  # Remove section headers
    text = text.strip()
    return text

In [10]:
# Function to fetch a Wikipedia article
def get_wikipedia_text(topic):
    page = wiki_wiki.page(topic)
    if not page.exists():
        return None

    text = clean_wiki_text(page.text)

    # Split text into chunks of 100-200 words
    sentences = text.split('. ')
    chunk = []
    dataset = []

    for sentence in sentences:
        chunk.append(sentence)
        word_count = len(' '.join(chunk).split())

        if 100 <= word_count <= 200:  # Ensure each chunk has 100-200 words
            dataset.append(' '.join(chunk))
            chunk = []  # Reset for next chunk

    return dataset

In [11]:
# Collect Wikipedia text samples
wikipedia_texts = []
for topic in topics:
    if len(wikipedia_texts) >= 2000:
        break  # Stop when we reach 2000 samples
    try:
        text_chunks = get_wikipedia_text(topic)
        if text_chunks:
            wikipedia_texts.extend(text_chunks)
    except Exception as e:
        print(f"Error fetching topic {topic}: {e}")

In [12]:
# Create a DataFrame
df_wikipedia = pd.DataFrame(wikipedia_texts, columns=["text"])
df_wikipedia["label"] = 0  # Label 0 for human-written text


In [13]:
# Save as CSV
df_wikipedia.to_csv("wikipedia_texts.csv", index=False)

In [15]:
print(f"Successfully collected {len(df_wikipedia)} Wikipedia samples!")

Successfully collected 1909 Wikipedia samples!


In [17]:
import requests
from bs4 import BeautifulSoup
import time

In [53]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

# Use the 61 collected URLs
article_urls = [
    "https://www.aljazeera.com/economy/2025/3/14/eu-parliament-rocked-by-corruption-investigation-linked-to-chinas-huawei",
    "https://www.aljazeera.com/opinions/2025/3/7/the-world-cannot-ignore-trumps-death-threat-to-the-people-of-gaza",
    "https://www.aljazeera.com/opinions/2025/3/8/a-tribute-to-um-adnan",
    "https://www.aljazeera.com/economy/2025/3/14/consumer-sentiment-plunges-as-us-frightened-of-sharply-higher-prices",
    "https://www.aljazeera.com/opinions/2025/3/9/america-needs-donald-trump-right-now-heres-why",
    "https://www.aljazeera.com/economy/2025/3/13/us-public-souring-on-trumps-handling-of-economy-polls-show",
    "https://www.aljazeera.com/news/2025/3/15/israel-kills-at-least-nine-palestinians-including-journalists-in-gaza",
    "https://www.aljazeera.com/economy/2025/3/11/even-as-us-slashes-jobs-it-is-the-calm-before-the-storm-economists-warn",
    "https://www.aljazeera.com/news/2025/3/15/syrians-mark-revolution-anniversary-with-roses-songs-and-tight-security",
    "https://www.aljazeera.com/features/2025/3/9/searching-for-africas-lost-world-war-soldiers-one-grave-at-a-time",
    "https://www.aljazeera.com/opinions/2025/3/12/arrests-of-columbia-pro-palestine-activists-will-not-save-israels-image",
    "https://www.aljazeera.com/features/2025/3/9/beauty-in-gaza-noors-tent-salon-in-the-rubble",
    "https://www.aljazeera.com/opinions/2025/3/6/a-new-muslim-and-african-ban-by-trump-can-and-must-be-prevented",
    "https://www.aljazeera.com/features/2025/3/14/is-trump-trying-to-pull-putin-away-from-china-and-can-it-work",
    "https://www.aljazeera.com/sports/2025/3/14/indian-premier-league-2025-axar-patel-named-delhi-capitals-captain",
    "https://www.aljazeera.com/news/2025/3/11/trump-says-imposing-50-percent-tariffs-on-canadian-steel-aluminium",
    "https://www.aljazeera.com/economy/2025/3/10/be-quiet-small-man-musk-clashes-with-polish-pm-over-starlink-in-ukraine",
    "https://www.aljazeera.com/sports/2025/3/13/australian-f1-gp-lewis-hamilton-itching-to-make-ferrari-debut",
    "https://www.aljazeera.com/sports/2025/3/16/pakistan-lose-first-t20-against-new-zealand-despite-khushdil-shah-heroics",
    "https://www.aljazeera.com/news/2025/3/15/ailing-pope-francis-works-on-signature-reform-from-hospital-as-he-recovers",
    "https://www.aljazeera.com/features/2025/3/12/a-rain-of-rockets-and-bullets-survivors-of-pakistans-train-hijacking",
    "https://www.aljazeera.com/economy/2025/3/14/meta-to-test-community-notes-using-algorithm-from-elon-musks-x",
    "https://www.aljazeera.com/opinions/2025/3/6/multilateralism-can-and-must-deliver",
    "https://www.aljazeera.com/news/2025/3/15/uks-starmer-says-coalition-to-beef-up-ukraine-security-in-any-peace-deal",
    "https://www.aljazeera.com/features/2025/3/13/palestinians-displaced-by-israeli-aggression-hit-new-levels-of-uncertainty",
    "https://www.aljazeera.com/economy/2025/3/12/bank-of-canada-cuts-interest-rates-warned-country-faces-new-crisis",
    "https://www.aljazeera.com/news/2025/3/13/trumps-steel-aluminium-tariffs-how-are-targeted-countries-responding",
    "https://www.aljazeera.com/news/2025/3/13/people-need-a-break-drc-conflict-reignites-dark-memories-of-congo-wars",
    "https://www.aljazeera.com/news/2025/3/15/huge-power-outage-in-cuba-leaves-millions-in-darkness",
    "https://www.aljazeera.com/opinions/2025/3/15/gazas-silent-epidemic",
    "https://www.aljazeera.com/opinions/2025/3/10/black-palestinian-solidarity-we-need-to-talk-about-what-happened",
    "https://www.aljazeera.com/economy/2025/3/13/elon-musk-wants-to-use-ai-to-run-us-govt-but-experts-say-very-bad-idea",
    "https://www.aljazeera.com/opinions/2025/3/13/trumps-tariffs-may-end-up-blowing-up-the-us-dollar-hegemony",
    "https://www.aljazeera.com/features/2025/3/10/we-just-want-peace-a-pacifist-community-amid-ethiopias-amhara-conflict",
    "https://www.aljazeera.com/economy/2025/3/11/bloodbath-us-stock-market-sheds-1-75tn-after-trumps-recession-remarks",
    "https://www.aljazeera.com/opinions/2025/3/13/trumps-usaid-freeze-must-serve-as-a-wake-up-call-for-africa",
    "https://www.aljazeera.com/news/2025/3/15/inch-by-inch-myanmar-rebels-close-in-on-key-military-base-in-chin-state",
    "https://www.aljazeera.com/news/2025/3/15/thousands-join-antigovernment-protest-in-serbias-capital-belgrade",
    "https://www.aljazeera.com/sports/2025/3/15/atletico-madrid-vs-barcelona-laliga-preview-teams-how-to-follow-stream",
    "https://www.aljazeera.com/news/2025/3/15/deadly-storms-kill-at-least-17-across-three-us-states",
    "https://www.aljazeera.com/sports/2025/3/14/alcarazs-indian-wells-three-peat-bid-on-track-with-draper-next-up",
    "https://www.aljazeera.com/economy/2025/3/14/us-stock-market-takes-another-tumble-as-trump-threatens-tariffs-on-wine",
    "https://www.aljazeera.com/news/2025/3/13/families-of-dutertes-drug-war-victims-grieve-seek-justice-in-philippines",
    "https://www.aljazeera.com/sports/2025/3/14/formula-one-south-africa-bids-to-return-f1-to-continent-where-when-how",
    "https://www.aljazeera.com/news/2025/3/11/why-china-isnt-as-worried-about-trumps-trade-war-as-in-2018",
    "https://www.aljazeera.com/news/2025/3/15/us-launches-major-air-strikes-on-houthi-targets-in-yemen-at-least-9-killed",
    "https://www.aljazeera.com/opinions/2025/3/7/the-real-reason-behind-trumps-tariffs-on-mexico-and-military",
    "https://www.aljazeera.com/news/2025/3/12/closest-target-why-is-donald-trump-so-focused-on-canada",
    "https://www.aljazeera.com/sports/2025/3/15/wpl-2025-final-mumbai-indians-claim-second-title-as-delhi-capitals-despair",
    "https://www.aljazeera.com/sports/2025/3/13/thunder-beat-celtics-to-clinch-nba-playoff-spot",
    "https://www.aljazeera.com/opinions/2025/3/12/trump-wants-to-deport-foreign-students-like-me-universities-must-defy-him",
    "https://www.aljazeera.com/sports/2025/3/14/fernandes-hits-back-at-ratcliffe-over-overpaid-jibe-at-manchester-united"
]


# Headers for requests
HEADERS = {"User-Agent": "Mozilla/5.0"}

def extract_article_text(url):
    """Extracts and cleans article text from an Al Jazeera news page."""
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code != 200:
            print(f"⚠️ Failed to fetch {url} (Status: {response.status_code})")
            return None

        soup = BeautifulSoup(response.text, "html.parser")

        # Locate the main article content
        article_body = soup.find("div", class_="wysiwyg wysiwyg--all-content")
        if not article_body:
            print(f"❌ Article body not found for {url}")
            return None

        paragraphs = article_body.find_all("p")
        text = " ".join([p.get_text() for p in paragraphs])

        # Clean text
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    except Exception as e:
        print(f"Error extracting {url}: {e}")
        return None

def split_into_samples(text, min_words=100, max_words=200):
    """Splits a long text into multiple samples of 100-200 words."""
    words = text.split()
    samples = []

    i = 0
    while i < len(words):
        length = min(max_words, len(words) - i)
        if length < min_words:
            break  # Stop if remaining text is too short
        sample = " ".join(words[i:i + length])
        samples.append(sample)
        i += length  # Move to next section

    return samples

# List to store samples
samples = []

# Process each URL
for url in article_urls:
    print(f"🔍 Scraping: {url}")
    article_text = extract_article_text(url)

    if article_text:
        text_samples = split_into_samples(article_text)
        samples.extend(text_samples)

    time.sleep(1)  # Be respectful with delays

# Create DataFrame
df = pd.DataFrame(samples, columns=["text"])
df["label"] = 0  # Human-written label

# Save to CSV
df.to_csv("aljazeera_news_articles.csv", index=False)

print(f"✅ Successfully collected {len(df)} samples!")

🔍 Scraping: https://www.aljazeera.com/economy/2025/3/14/eu-parliament-rocked-by-corruption-investigation-linked-to-chinas-huawei
🔍 Scraping: https://www.aljazeera.com/opinions/2025/3/7/the-world-cannot-ignore-trumps-death-threat-to-the-people-of-gaza
🔍 Scraping: https://www.aljazeera.com/opinions/2025/3/8/a-tribute-to-um-adnan
🔍 Scraping: https://www.aljazeera.com/economy/2025/3/14/consumer-sentiment-plunges-as-us-frightened-of-sharply-higher-prices
🔍 Scraping: https://www.aljazeera.com/opinions/2025/3/9/america-needs-donald-trump-right-now-heres-why
🔍 Scraping: https://www.aljazeera.com/economy/2025/3/13/us-public-souring-on-trumps-handling-of-economy-polls-show
🔍 Scraping: https://www.aljazeera.com/news/2025/3/15/israel-kills-at-least-nine-palestinians-including-journalists-in-gaza
🔍 Scraping: https://www.aljazeera.com/economy/2025/3/11/even-as-us-slashes-jobs-it-is-the-calm-before-the-storm-economists-warn
🔍 Scraping: https://www.aljazeera.com/news/2025/3/15/syrians-mark-revolution-

### Books

In [66]:
import requests
from bs4 import BeautifulSoup
import csv
import random
import time

# List of popular books by ID (from Gutenberg)
book_ids = [
    1342, 84, 1080, 2600, 6130, 1661, 11, 5200, 2701, 98, 174, 43,
    25344, 16, 1400, 74, 1952, 2852, 829, 1250, 1727, 345, 730, 30254, 1260
]

# Function to extract book text from Gutenberg
def extract_gutenberg_text(book_id):
    """Download and extract book text from Project Gutenberg"""
    try:
        url = f"https://www.gutenberg.org/files/{book_id}/{book_id}-0.txt"
        response = requests.get(url)
        response.raise_for_status()

        text = response.text
        start_index = text.find("*** START OF THIS PROJECT GUTENBERG EBOOK")
        end_index = text.find("*** END OF THIS PROJECT GUTENBERG EBOOK")

        if start_index != -1 and end_index != -1:
            text = text[start_index:end_index]
        elif start_index != -1:
            text = text[start_index:]
        elif end_index != -1:
            text = text[:end_index]

        return text if len(text.split()) > 100 else None  # Ensure valid text

    except Exception as e:
        print(f"❌ Error extracting book {book_id}: {e}")
        return None

# Function to split text into 100-200 word samples
def split_into_samples(text, min_words=100, max_words=200):
    """Split text into multiple samples of 100-200 words"""
    words = text.split()
    samples = []

    while len(words) >= min_words:
        sample_length = random.randint(min_words, min(max_words, len(words)))
        sample = " ".join(words[:sample_length])
        samples.append(sample)
        words = words[sample_length:]  # Remove used words

    return samples

# Collect 3,000 samples
samples = []
for book_id in book_ids:
    print(f"📖 Extracting book ID: {book_id}")
    text = extract_gutenberg_text(book_id)

    if text:
        book_samples = split_into_samples(text, min_words=100, max_words=200)
        samples.extend(book_samples)

    if len(samples) >= 3000:
        break  # Stop when we reach 3,000 samples

    time.sleep(1)  # Respectful delay

# Limit to exactly 3,000 samples
samples = samples[:3000]

# Save to CSV
filename = "gutenberg_samples.csv"
with open(filename, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["text", "label"])
    writer.writerows([[s, 0] for s in samples])  # Label 0 for human-written text

# Completion message
print(f"\n✅ Successfully saved 3,000 samples to {filename}!")


📖 Extracting book ID: 1342
📖 Extracting book ID: 84
📖 Extracting book ID: 1080
📖 Extracting book ID: 2600

✅ Successfully saved 3,000 samples to gutenberg_samples.csv!


**MACHINE GENERATED TEXT**

In [68]:
from transformers import pipeline
import csv
import random
import time

In [2]:
from huggingface_hub import login
login()

In [ ]:
import pandas as pd
import random
from transformers import pipeline

# Load GPT-2 model
generator = pipeline("text-generation", model="gpt2")

# Define categories and templates for prompts
categories = ["news", "story", "essay", "science", "history", "opinion", "fantasy", "business", "technology", "politics", "environment", "health", "sports", "culture", "travel", "food", "art", "literature", "economics", "education", "philosophy", "psychology", "sociology", "design", "film", "music", "gaming", "fashion", "law", "religion", "human rights", "investigative", "documentary", "analysis", "review", "interview", "profiles", "editorial", "commentary", "data journalism", "multimedia", "longform", "short story", "poetry", "satire", "feature", "development", "innovation", "geopolitics", "climate", "sustainability", "space", "archeology", "anthropology"]

templates = [
    "Write a {category} article about {topic}.",
    "Explain the significance of {topic} in {category}.",
    "Tell a short {category} story about {topic}.",
    "What are the future implications of {topic} in {category}?",
    "Describe a day in the life of someone in {category} focusing on {topic}.",
    "Give an in-depth analysis of {topic} within the field of {category}.",
    "Summarize the history and evolution of {topic} in {category}.",
    "What are the ethical concerns related to {topic} in {category}?",
    "How is {topic} expected to change {category} in the next decade?",
    "Compare and contrast different viewpoints on {topic} in {category}."
]

topics = [
    # Science & Technology
    "quantum computing", "dark matter", "human cloning", "AI ethics",
    "nanotechnology", "renewable energy", "5G networks", "data privacy",
    "gene editing", "future of robotics", "electric vehicles", "space tourism",

    # History & Politics
    "World War I", "Cold War", "rise and fall of empires", "industrial revolution",
    "ancient Egyptian civilization", "civil rights movements", "history of democracy",
    "famous historical betrayals", "nuclear arms race", "colonialism and its effects",

    # Economy & Business
    "cryptocurrency regulation", "stock market crashes", "startup culture",
    "future of freelancing", "automation in the workplace", "universal basic income",
    "the future of e-commerce", "gig economy", "the impact of inflation",

    # Health & Society
    "mental health awareness", "work-life balance", "veganism trends",
    "remote work culture", "rise of minimalist lifestyle", "effects of social media",
    "gender equality", "challenges of modern parenting", "impact of aging populations",

    # Environment & Nature
    "climate change policies", "ocean pollution", "wildlife conservation",
    "deforestation crisis", "future of agriculture", "urban sustainability",
    "eco-friendly architecture", "impact of wildfires",

    # Space & Exploration
    "colonizing Mars", "life beyond Earth", "black holes",
    "exoplanets", "the Fermi paradox", "space mining",

    # Random Topics
    "philosophy of free will", "future of education", "importance of reading books",
    "mysteries of the deep sea", "role of luck in success", "sports psychology",
    "music and brain function", "fashion trends over time", "history of magic"
]


def generate_prompts(num_prompts=1000):
    prompts = []
    for _ in range(num_prompts):
        category = random.choice(categories)
        topic = random.choice(topics)
        prompt_template = random.choice(templates)
        prompt = prompt_template.format(category=category, topic=topic)
        prompts.append(prompt)
    return prompts

# Generate prompts
num_samples = 6000 ###CHANGE
batch_size = 20  # Save every 20 samples ###CHANGE
samples = []
prompts = generate_prompts(num_samples)

# Generate and save text
for idx, prompt in enumerate(prompts):
    generated_text = generator(
    prompt,
    max_length=150,
    num_return_sequences=1,
    truncation=True,
    pad_token_id=50256  # EOS token to remove the warning
)[0]["generated_text"]

    samples.append({"text": generated_text, "label": 1})  # Label 1 for machine-generated text

    # Save every 20th samples
    if (idx + 1) % batch_size == 0:
        df = pd.DataFrame(samples)
        df.to_csv("machine_generated_text_final.csv", index=False, mode="a", header=(idx == 0)) ####CHANGE
        print(f"✅ Saved {idx + 1} samples...")

print("✅ Successfully generated and saved 6000 machine-generated text samples!")


Device set to use cpu


✅ Saved 20 samples...
✅ Saved 40 samples...
✅ Saved 60 samples...
✅ Saved 80 samples...
✅ Saved 100 samples...
✅ Saved 120 samples...
✅ Saved 140 samples...
✅ Saved 160 samples...
✅ Saved 180 samples...
✅ Saved 200 samples...
✅ Saved 220 samples...
✅ Saved 240 samples...
✅ Saved 260 samples...
✅ Saved 280 samples...
✅ Saved 300 samples...
✅ Saved 320 samples...
✅ Saved 340 samples...
✅ Saved 360 samples...
✅ Saved 380 samples...
✅ Saved 400 samples...
✅ Saved 420 samples...
✅ Saved 440 samples...
✅ Saved 460 samples...
✅ Saved 480 samples...
✅ Saved 500 samples...
✅ Saved 520 samples...
✅ Saved 540 samples...
✅ Saved 560 samples...
✅ Saved 580 samples...
✅ Saved 600 samples...
✅ Saved 620 samples...
✅ Saved 640 samples...
✅ Saved 660 samples...
✅ Saved 680 samples...
✅ Saved 700 samples...
✅ Saved 720 samples...
✅ Saved 740 samples...
✅ Saved 760 samples...
✅ Saved 780 samples...
✅ Saved 800 samples...
✅ Saved 820 samples...
✅ Saved 840 samples...
✅ Saved 860 samples...
✅ Saved 880 sam

In [38]:
import pandas as pd
import re
from bs4 import BeautifulSoup

# Load the dataset
df = pd.read_csv("machine_generated_text_final.csv", header=None, names=["text", "label"])

In [39]:
# Ensure "text" is a string before processing
df["text"] = df["text"].astype(str)

In [40]:
len(df)

781200

In [41]:
# Remove extra spaces & line breaks
df["text"] = df["text"].apply(lambda x: " ".join(str(x).split()))

In [42]:
len(df)

781200

In [43]:
# Remove non-alphanumeric characters (except punctuation)
import re
df["text"] = df["text"].apply(lambda x: re.sub(r"[^a-zA-Z0-9.,!? ]+", "", x))

In [44]:
len(df)

781200

In [45]:
# Remove exact duplicates
df = df.drop_duplicates()
print("After removing duplicates:", len(df))

After removing duplicates: 5580


In [46]:
print(df.head())

                                                text  label
0  How is space tourism expected to change educat...      1
1  Write a sustainability article about gig econo...      1
2  What are the ethical concerns related to myste...      1
3  Explain the significance of space mining in re...      1
4  Tell a short space story about 5G networks. We...      1


In [72]:
df = df[df["text"].apply(lambda x: len(x.split()) >= 26)]

In [73]:
len(df)

5092

In [74]:
# Save cleaned file
df.to_csv("machine_generated_text_cleaned.csv", index=False)

print("Cleaning complete. Final records:", len(df))

Cleaning complete. Final records: 5092


Cleaning for Human text

In [75]:
import pandas as pd
import re

# Load the CSV files
wikipedia_df = pd.read_csv("wikipedia_texts.csv")
aljazeera_df = pd.read_csv("aljazeera_news_articles.csv")
gutenberg_df = pd.read_csv("gutenberg_samples.csv")

In [78]:
print(wikipedia_df.head())
print(aljazeera_df.head())
print(gutenberg_df.head())

                                                text  label
0  Artificial intelligence  refers to the capabil...      0
1  However, many AI applications are not perceive...      0
2  AI also draws upon psychology, linguistics, ph...      0
3  The emergence of advanced generative AI in the...      0
4  Reasoning and problem-solving Early researcher...      0
                                                text  label
0  Police have arrested several people as part of...      0
1  added. Prosecutors said they could not disclos...      0
2  “To the People of Gaza: A beautiful Future awa...      0
3  To the grieving mothers who have buried their ...      0
4  who claims to stand for peace. This is not jus...      0
                                                text  label
0  *** START OF THE PROJECT GUTENBERG EBOOK 1342 ...      0
1  others who are loved “by allowance” by convent...      0
2  faint in tone, and not enthralling in interest...      0
3  of its composition seen rather more o

In [79]:
# Combine all data
human_df = pd.concat([wikipedia_df, aljazeera_df, gutenberg_df], ignore_index=True)

In [80]:
print(human_df.head())

                                                text  label
0  Artificial intelligence  refers to the capabil...      0
1  However, many AI applications are not perceive...      0
2  AI also draws upon psychology, linguistics, ph...      0
3  The emergence of advanced generative AI in the...      0
4  Reasoning and problem-solving Early researcher...      0


In [81]:
len(human_df)

5152

In [82]:
# Shuffle the data after merging
human_df = human_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [83]:
print(human_df.head())

                                                text  label
0  Political science The application of game theo...      0
1  The arms are dusty regions of star formation w...      0
2  so; for, besides the immediate embarrassment, ...      0
3  However, there remain difficult questions abou...      0
4  charity in a house where such things could be ...      0


In [84]:
# Remove extra spaces and line breaks
human_df["text"] = human_df["text"].apply(lambda x: " ".join(str(x).split()))

In [85]:
# Remove non-alphanumeric characters (except common punctuation)
human_df["text"] = human_df["text"].apply(lambda x: re.sub(r"[^a-zA-Z0-9.,!? ]+", "", x))

In [86]:
# Check the minimum word length in the text column
min_word_count = human_df["text"].apply(lambda x: len(str(x).split())).min()
print(f"Minimum word count in the text column: {min_word_count}")

Minimum word count in the text column: 74


In [88]:
# Check the minimum word length in the text column
min_word_count = human_df["text"].apply(lambda x: len(str(x).split())).max()
print(f"Max word count in the text column: {min_word_count}")

Max word count in the text column: 200


In [89]:
# Save to a new file
human_df.to_csv("human_written_texts_cleaned.csv", index=False)

COMBINING THE TWO DATASETS (MACHINE AND HUMAN)

In [90]:
# Load the cleaned datasets
human_df = pd.read_csv("human_written_texts_cleaned.csv")
machine_df = pd.read_csv("machine_generated_text_cleaned.csv")

In [91]:
print("Human Data Labels - Min:", human_df["label"].min(), "Max:", human_df["label"].max())
print("Machine Data Labels - Min:", machine_df["label"].min(), "Max:", machine_df["label"].max())

Human Data Labels - Min: 0 Max: 0
Machine Data Labels - Min: 1 Max: 1


Human data with Label : 0 and Machine data with Label: 1

In [92]:
# Combine both datasets
combined_df = pd.concat([human_df, machine_df], ignore_index=True)

In [93]:
# Shuffle the dataset to mix human and machine-generated text
combined_df = combined_df.sample(frac=1, random_state=72).reset_index(drop=True)

In [94]:
print(combined_df.head())

                                                text  label
0  In the Vigenre cipher, a polyalphabetic cipher...      0
1  the carriage came to the door, my uncle was ca...      0
2  some partiality, especially on Lydias side, bu...      0
3  her there, Natsha ran to the nursery, but Snya...      0
4  What are the ethical concerns related to unive...      1


In [95]:
len(combined_df)

10244

In [96]:
# Save the final dataset
combined_df.to_csv("final_combined_dataset.csv", index=False)